In [16]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time 
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
import xgboost as xgb
import math 
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from geopy.distance import great_circle
from math import sin, cos, sqrt, atan2


In [24]:
def drop_virgule(x):
    try:return x.split('.')[0]
    except:return x

def distance_to_mid_center(lat, lon):
    berlin_center = (52.5200, 13.4050)
    accommodation = (lat, lon)
    return great_circle(berlin_center, accommodation).km


def processing(df):
    df.drop(columns=columns_to_drop,inplace=True) #specified irrelevant columns 
    
    df.dropna(subset=['Price'], inplace=True)    #drop price missing (9rows)
    
    df['Listing ID']=df['Listing ID'].apply(drop_virgule)              # Drop the '.0' in IDs and Postal Code
    df['Host ID']=df['Host ID'].apply(drop_virgule)
    df['Postal Code']=df['Postal Code'].apply(drop_virgule)
    
    df['Host Since']=pd.to_datetime(df['Host Since'])                  # Convert values to datetime
    df['First Review']=pd.to_datetime(df['First Review'])
    df['Last Review']=pd.to_datetime(df['Last Review'])
    
    df['Accomodates'].replace('*', np.nan,inplace=True)                 # Put NaN in missing values where we have *
    df['Bathrooms'].replace('*', np.nan,inplace=True)
    df['Bedrooms'].replace('*', np.nan,inplace=True)
    df['Beds'].replace('*', np.nan,inplace=True)
    df['Guests Included'].replace('*', np.nan,inplace=True)
    df['Min Nights'].replace('*', np.nan,inplace=True)
    df['Postal Code'].replace('*', np.nan,inplace=True)
    df['neighbourhood'].replace('*', np.nan,inplace=True)
    df['Property Type'].replace('*', np.nan,inplace=True)
    df['Host Response Rate'].replace('*', np.nan,inplace=True)

    #from 98% to 0.98
    df['Host Response Rate'].replace('nan', np.nan,inplace=True)
    df['Host Response Rate']=df[~df['Host Response Rate'].isnull()]["Host Response Rate"].str.split('%').apply(lambda x:float(x[0])/100)

    df['Accomodates'] = df['Accomodates'].astype('float')              # Convert values to float
    df['Bathrooms'] = df['Bathrooms'].astype('float')
    df['Bedrooms'] = df['Bedrooms'].astype('float')
    df['Beds'] = df['Beds'].astype('float')
    df['Guests Included'] = df['Guests Included'].astype('float')
    df['Min Nights'] = df['Min Nights'].astype('float') 
    
    df.columns = df.columns.str.lower().str.replace(' ','_')           # Rename the columns with '_' instead of ' '

    #distance from center of berlin
    df['distance_to_midcenter'] = df.apply(lambda x: distance_to_mid_center(x.latitude, x.longitude), axis=1)
    for lieu in centres_berlins:                                       # Add distances 
        df = add_distance_feature(df, lieu)

    return df

def processing_2(df):
    
    numerical_columns = df.select_dtypes(exclude=object).columns   # numeric columns names
    categorical_columns = df.select_dtypes(include=object).columns # categorical columns names
    
    df_cleaned_num = df[numerical_columns]    # dataframe with numeric columns only
    df_cleaned_cat = df[categorical_columns]  # dataframe with categorical columns only
    
    #standardisation for numeric columns
    #scalerx = MinMaxScaler() 
    #df_scaled=pd.DataFrame(scalerx.fit_transform(df_cleaned_num), columns = numerical_columns)
    
    #one hot encoder for categorical columns
    df_encoded=pd.DataFrame() 
    for cat in categorical_columns:
        df_temp = pd.get_dummies(df_cleaned_cat[cat], prefix=cat)
        df_encoded=pd.concat([df_temp, df_encoded], axis=1)
    
    #concatenation of numeric and categorical dataframes
    df_cleaned_num.reset_index(drop=True, inplace=True)
    df_encoded.reset_index(drop=True, inplace=True)
    df_final=pd.concat([df_cleaned_num, df_encoded], axis=1) 
    
    #KNN imputer (n=5 ????)
    imputer = KNNImputer(n_neighbors=5)
    df_final = pd.DataFrame(imputer.fit_transform(df_final),columns = df_final.columns)
    
    return df_final 


In [2]:

#liste des lieux emblématiques de Berlin [nom,lat,long]
centres_berlins = [["charlottenburg", 52.516602, 13.304105],
                   ["kreuzberg", 52.498605, 13.391799],
                   ["wedding", 52.561559, 13.35002],
                   ["mitte", 52.531677, 13.381777],
                   ["mariendorf", 52.4333316, 13.3833318],
                   ["tegel", 52.558833, 13.288437], 
                   ["tempelhof", 52.472160, 13.370287],
                   ["spandau", 52.534080, 13.181716],
                   ["schöneberg", 52.497161, 13.346865],
                   ["wilmersdorf",  52.48333, 13.31667],
                   ["biesdorf", 52.508429, 13.563317],
                   ["moabit", 52.530832, 13.345876],
                   ["britz", 52.45, 13.433333],
                   ["neukölln", 52.440771, 13.444507],
                   ["dahlem", 52.466562, 13.300082], 
                   ["tiergarten", 52.51449, 13.350091],
                   ["hellersdorf", 52.536107, 13.604973],
                   ["prenzlauer_berg", 52.550113, 13.423125], 
                   ["friedrichshain", 52.515816, 13.454293],
                   ["reinickendorf", 52.566667, 13.333333],
                   ["friedrichsfelde", 52.503664652, 13.507664636],
                   ["friedenau", 52.47133, 13.32813],
                   ["gesundbrunnen", 52.548611, 13.390278],
                   ["charlottenbourg_nord", 52.53048, 13.29371],
                   ["hansaviertel", 52.5166646, 13.33666532],
                   ["haselhorst", 52.54409, 13.23743],
                   ["gropiusstadt", 52.425, 13.46667],
                   ["westend", 52.5166646, 13.2833322],
                   ["wittenau", 52.592455, 13.329694],
                   ["zehlendorf", 52.435077, 13.260425],
                   ["lichtenberg", 52.534306, 13.502326],
                   ["pankow", 52.592879, 13.431700],
                   ["steglitz", 52.453096, 13.331171],
                   ["siemenstadt", 52.537664516, 13.257832302],
                   ["rudow", 52.402310, 13.509220],
                   ["kaulsdorf", 52.506512, 13.593946]]

In [9]:
def add_distance_feature(df, lieu):
    name_column = "distance_" + str(lieu[0])
    df[name_column] = df.apply(lambda x : distance(x['latitude'], x['longitude'], lieu[1], lieu[2]), axis = 1)
    return df
def distance(lat1,lon1,lat2,lon2):
    R = 6373.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

In [25]:
path='train_airbnb_berlin.csv'
my_sep,my_encoding=',','utf-8'
columns_to_drop=['Listing Name','Host Name','City','Country Code','Country']
types={'Listing ID':'str','Host ID':'str','Postal Code':'str'}
data=pd.read_csv(path,sep=my_sep,encoding=my_encoding,dtype=types)

#processing 1: drop the irrelevant columns / replace '*' values by nan / standardisation of columns name
df=processing(data)

df.drop(columns=['host_since','last_review','first_review','square_feet',
                 'business_travel_ready','host_id','listing_id','host_since',
                 'first_review','last_review','neighborhood_group'],inplace=True)

# droping outlighers for prices >300
df = df[~(df['price'] > 300)]

#droping rows where there is more than 3 nan 
#from dataviz we saw that it was the best option
df=df[df.isnull().sum(1)<4]

#processing 2: standardisation / one-hot-encoder / KNN imputer
df=processing_2(df)
print(df.shape)
df

(12622, 264)


,host_response_rate,latitude,longitude,accomodates,bathrooms,bedrooms,beds,guests_included,min_nights,reviews,...,neighbourhood_Wilhelmstadt,neighbourhood_Wilmersdorf,neighbourhood_Wittenau,neighbourhood_Zehlendorf,is_superhost_f,is_superhost_t,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour
0,0.914,52.54652,13.41792,2.0,1.0,1.0,1.0,1.0,2.0,6.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.942,52.56512,13.42214,2.0,1.0,2.0,2.0,2.0,7.0,6.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.000,52.54741,13.42521,3.0,1.0,1.0,2.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.950,52.50958,13.45144,2.0,1.0,1.0,1.0,1.0,2.0,4.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.000,52.44826,13.40608,3.0,1.0,1.0,2.0,2.0,1.0,10.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12617,0.980,52.52759,13.33738,1.0,1.0,1.0,1.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
12618,0.634,52.52638,13.39698,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
12619,0.330,52.49016,13.39241,2.0,1.5,1.0,1.0,1.0,6.0,11.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
12620,0.940,52.47586,13.45362,8.0,2.0,3.0,5.0,1.0,1.0,4.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [26]:
X=df.drop(columns=['price'])
y=df['price'].copy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
import xgboost as xgb

regressor = xgb.XGBRegressor(
    n_estimators=150,
    reg_lambda=2,
    gamma=1,
    max_depth=3
)
regressor.fit(X_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=1, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=150, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=2, ...)

In [27]:
import math 
from sklearn import metrics

y_pred = regressor.predict(X_test)
print('rmse ',math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('r2   ',metrics.r2_score(y_test, y_pred))

rmse  28.460277010008472
r2    0.5129758012625798
